# SQL para preparação de dados

<p><img src = https://www.dataquest.io/wp-content/uploads/2021/11/why-sql-consumes-so-much-memory-header.webp width = 500></p>



## Montagem de dados
### Conectando tabelas usando JOIN
Discutimos, anteriomente, como podemos consultar dados de uma tabela. No entanto, na maioria das vezes, os dados nos quais você está interessado estão espalhados por várias tabelas. Felizmente, o SQL tem métodos para reunir tabelas relacionadas usando a palavra-chave **JOIN**.

Para ilustrar, vejamos duas tabelas em nosso banco de dados – dealerships (concessionárias) e **salespeople** (vendedores). Na tabela **salespeople**, observamos que temos uma coluna chamada ***dealership_id***. Esta coluna *dealership_id* é uma referência direta à coluna *dealership_id* na tabela de concessionárias. Quando a tabela A tem uma coluna que faz referência à chave primária da tabela B, diz-se que a coluna é uma chave estrangeira para a tabela A. Nesse caso, a coluna *dealership_id* em **salespeople** é uma chave estrangeira para a tabela de concessionárias.
> ##### Observação
> As chaves estrangeiras também podem ser adicionadas como uma restrição de coluna (constraint column) a uma tabela para melhorar a integridade dos dados, garantindo que a chave estrangeira nunca contenha um valor que não possa ser encontrado na tabela referenciada. Essa propriedade de dados é conhecida como integridade referencial. Adicionar restrições de chave estrangeira também pode ajudar a melhorar o desempenho em alguns bancos de dados. As restrições de chave estrangeira estão além do escopo deste módulo e, na maioria dos casos, os engenheiros de dados e os administradores de banco de dados da sua empresa lidarão com esses detalhes. Você pode aprender mais sobre restrições de chave estrangeira na documentação do PostgreSQL no seguinte link:
https://www.postgresql.org/docs/9.4/tutorial-fk.html.

Como essas tabelas estão relacionadas, você pode realizar algumas análises interessantes com essas duas tabelas. Por exemplo, você pode estar interessado em determinar quais vendedores trabalham em uma concessionária na Califórnia. Uma maneira de recuperar essas informações é primeiro consultar quais concessionárias estão localizadas na Califórnia usando a seguinte consulta:

In [2]:
import pandas as pd
import psycopg2 as pg

# Realizando a conexão local no Banco de dados (SQLDA)
connection = pg.connect(user="postgres",
                                  password="admin",
                                  host="127.0.0.1",
                                  port="5432",
                                  database="sqlda")
# Query a ser realizada

sql = """
      SELECT *
      FROM dealerships
      WHERE state = 'CA'
      """
df = pd.read_sql_query(con=connection, sql=sql)
df

,dealership_id,street_address,city,state,postal_code,latitude,longitude,date_opened,date_closed
0,2,808 South Hobart Boulevard,Los Angeles,CA,90005,34.057754,-118.305423,2014-06-01,None
1,5,2210 Bunker Hill Drive,San Mateo,CA,94402,37.524487,-122.343609,2014-06-01,None


Agora que você sabe que as duas únicas concessionárias na Califórnia têm IDs 2 e 5, respectivamente, você pode consultar a tabela de vendedores da seguinte maneira:

In [4]:
sql = """
      SELECT *
      FROM salespeople
      WHERE dealership_id in (2, 5)
      ORDER BY 1;
      """
df = pd.read_sql_query(con=connection, sql=sql)
df.head()

,salesperson_id,dealership_id,title,first_name,last_name,suffix,username,gender,hire_date,termination_date
0,23,2,None,Beauregard,Peschke,None,bpeschkem,Male,2018-09-12,None
1,51,5,None,Lanette,Gerriessen,None,lgerriessen1e,Female,2018-06-24,None
2,57,5,None,Spense,Pithcock,None,spithcock1k,Male,2017-12-15,None
3,61,5,None,Ludvig,Baynam,None,lbaynam1o,Male,2016-08-25,None
4,62,2,None,Carroll,Pudan,None,cpudan1p,Female,2016-05-17,None


Embora esse método forneça os resultados desejados, é tedioso realizar duas consultas para obter esses resultados. O que facilitaria essa consulta seria adicionar de alguma forma as informações da tabela de concessionárias à tabela de vendedores e depois filtrar por usuários na Califórnia. SQL fornece tal ferramenta com a cláusula **JOIN**. A cláusula **JOIN** é uma cláusula SQL que permite a um usuário unir uma ou mais tabelas com base em condições distintas.